In [3]:
import torch
from torch_geometric.data import InMemoryDataset
import os.path as osp

In [ ]:
# The full wico graph with every induced subgraph compiled together
class FullWICO(Dataset):
    
    def __init__(self, root='/Users/maxperozek/GNN-research/data_pro/data/wico',
                 transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform)
        
    @property
    def raw_file_names(self):
        return ['filename.pt']

    @property
    def processed_file_names(self):
        # single graph
        return ['data_0.pt']

    def process(self):
        RAW_DATA_DIR = 'path_of_dataset/'

        idx = 0
        # for each list of items...
        for raw_path in self.raw_file_names:
            # Read data from `raw_path`
            data_list = torch.load(self.RAW_DATA_DIR+raw_path)
            for graph in data_list:
                data = graph
                if self.pre_filter is not None and not self.pre_filter(data):
                    continue

                if self.pre_transform is not None:
                    data = self.pre_transform(data)

                torch.save(data, osp.join(self.processed_dir, f'data_{idx}.pt'))
                idx += 1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data


In [ ]:
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        # Download to `self.raw_dir`.
        download_url(url, self.raw_dir)
        ...

    def process(self):
        # Read data into huge `Data` list.
        data_list = [...]

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])